<a href="https://colab.research.google.com/github/Max-FM/IAA-Social-Distancing/blob/master/retinanet_object_detector_stanford_drone_satellite_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Detecting Pedestrians in Satellite Images

Refer to [this](https://towardsdatascience.com/pedestrian-detection-in-aerial-images-using-retinanet-9053e8a72c6) article for the required data to run this notebook.

This notebook was adapted from [this](https://medium.com/@ije_good/object-detection-on-satellite-imagery-using-retinanet-part-1-training-e589975afbd5) article.

**Warning:** For some unknown reason the prediction/detection function has stopped working properly (likely due to a package update). Feel free to use this notebook as a template but it may no longer work properly out of the box.

**Disclaimer:** Satellite images are not publicly available in the GitHub repository in order to avoid potential legal issues. The images used in this notebook are available internally to other researchers at the University of Portsmouth [here](https://drive.google.com/drive/folders/1px4AYAmkRHoC3Zic3fnMykueWyFkhSK3?usp=sharing). Access is restricted to external persons and all external access requests will be denied. Should the user wish to acquire the images themselves, the corresponding shapefiles are publicly available in the repository. I suggest uploading the images to Google Drive and mounting using the cell below.

In [ ]:
%tensorflow_version 1.x

In [ ]:
#Edit filepaths as required.

base_dir = '/content'
drive_dir = '/content/drive/My Drive/Social Distancing Project'
sat_dir = f'{drive_dir}/Satellite Images/DigitalGlobe (Samples)'
drone_dir = f'{drive_dir}/Object Detection/Stanford Drone Subset'
repo_dir = f'{base_dir}/keras-retinanet' 

label_dir = f'{drone_dir}/labels.csv'
test_img_dir = f'{sat_dir}/Tripoli/Tripoli_Map-Ready_Ortho_30cm/055675519050/055675519050_01_P001_MUL/JPEG/'

'/content/drive/My Drive/Social Distancing Project/Satellite Images/DigitalGlobe (Samples)/Tripoli/Tripoli_Map-Ready_Ortho_30cm/055675519050/055675519050_01_P001_MUL/JPEG/'

###Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Clone Keras Retinanet Repo

In [ ]:
repo_url = 'https://github.com/fizyr/keras-retinanet'

!git clone '{repo_url}' # clone repo

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [ ]:
# from /content/keras-retinanet
%cd '{repo_dir}'
!pip install .
!python setup.py build_ext --inplace

%cd ..

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=171477 sha256=1855e29a3da6fbf529932c63f4f727ae9d8047a554fe839680272053bf01c576
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
Successfully built keras-retinanet
  Found existing installation: keras-retinanet 0.5.1
    Uninstalling keras-retinanet-0.5.1:
      Successfully uninstalled keras-retinanet-0.5.1
running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.6/keras_retinanet/utils/compute_overlap.cpython-36m-x86_64-linux-gnu.so -> keras_retinanet/utils
/content


## Clone ije_retinanet Repo


In [ ]:
ije_repo_url = 'https://github.com/ijeism/ije_retinanet'
ije_repo_dir = f'{base_dir}/ije_retinanet'

!git clone '{ije_repo_url}' # clone repo

fatal: destination path 'ije_retinanet' already exists and is not an empty directory.


## Generating predictions from pretrained model

In [ ]:
inference_model = f'{drone_dir}/resnet50_csv_12_inference_priya.h5'
out_dir = f'{test_img_dir}/output'

# create output directory
!mkdir '{out_dir}'

mkdir: cannot create directory ‘/content/drive/My Drive/Social Distancing Project/Satellite Images/DigitalGlobe (Samples)/Tripoli/Tripoli_Map-Ready_Ortho_30cm/055675519050/055675519050_01_P001_MUL/JPEG//output’: File exists


In [ ]:
# generate prediction files
!python '{base_dir}/ije_retinanet/image_inference_write.py' \
-i '{test_img_dir}' \
-t 0.6 \
-m '{inference_model}' \
-o  '{out_dir}' \
-l '{label_dir}'

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-09-01 13:14:18.733247: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-01 13:14:18.751037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-01 13:14:18.751808: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-09-01 13:14:18.752137: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-01 13:14:18.754427: I tensorflow/stream_executor/platform/default/dso_loader.cc

In [ ]:
# combine all csv files into one
import glob
import pandas as pd

columns = ['path', 'confidence' , 'ymin', 'xmin', 'ymax', 'xmax', 'class']
df_list = [pd.read_csv(csv, names=columns) for csv in glob.glob(f'{out_dir}/*.csv')]
df_final = pd.concat(df_list)
df_final.to_csv(f'{drive_dir}/out.csv', index=False)

display(df_final['class'].value_counts(), df_final.head())


Pedestrian    7
Name: class, dtype: int64

,path,confidence,ymin,xmin,ymax,xmax,class
0,/content/drive/My Drive/Social Distancing Proj...,0.709590,37.841354,103.900610,45.567043,111.32725,Pedestrian
0,/content/drive/My Drive/Social Distancing Proj...,0.679012,148.929870,190.937230,161.328190,199.43745,Pedestrian
0,/content/drive/My Drive/Social Distancing Proj...,0.621681,92.977260,131.157230,105.284950,138.25255,Pedestrian
0,/content/drive/My Drive/Social Distancing Proj...,0.744258,189.152680,37.463245,197.917970,46.43148,Pedestrian
0,/content/drive/My Drive/Social Distancing Proj...,0.619614,65.782585,136.130340,72.416190,144.08875,Pedestrian


## Generate detections on test images

In [ ]:
!mkdir '{out_dir}/imgs'

# generate detections on images
!python '{base_dir}/ije_retinanet/image_inference_print.py' \
-i '{test_img_dir}' \
-t 0.6 \
-m  '{inference_model}' \
-o '{out_dir}/imgs'

mkdir: cannot create directory ‘/content/drive/My Drive/Social Distancing Project/Satellite Images/DigitalGlobe (Samples)/Tripoli/Tripoli_Map-Ready_Ortho_30cm/055675519050/055675519050_01_P001_MUL/JPEG//output/imgs’: File exists
Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-09-01 12:49:34.072934: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-01 12:49:34.090169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-01 12:49:34.090937: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020